In [1]:
# Import Block
import numpy as np
from rdkit import Chem
from openbabel import pybel
from openff.toolkit.topology import Molecule,Topology
from openff.toolkit.utils import RDKitToolkitWrapper
import openmm
from simtk.openmm import app
from openmm import unit
from openmm.app import PDBFile
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmm.vec3 import Vec3
from scipy.optimize import minimize
import warnings
warnings.simplefilter("ignore")


In [2]:
# Wrapper and FF setup
# Use RDKit wrapper
rdktkw = RDKitToolkitWrapper()

# Loading setup parameters
forcefield = ForceField('openff-2.0.0.offxml')

In [3]:
# load pdb with one copy of pdb file
#off_mol = Molecule.from_pdb_and_smiles('7101899.pdb', "CC1=CN=C(C(=C1OC)C)C[S@@](=O)C2=NC3=C(N2)C=C(C=C3)OC")
off_mol = Molecule.from_pdb_and_smiles('1100249.pdb', "O=C1N([C@H]2C=C[C@@H]1[C@H]1C[C@@H]2C=C1)C(=O)O[C@H]1[C@@H](CC[C@H](C1)C)C(C)C")



In [4]:
# load supercell pdb file (2x2x2) into topology
pdb_file = PDBFile('1100249_supercell.pdb')
off_top = Topology.from_openmm(pdb_file.topology, [off_mol])
# Create MD simulation inputs
system = forcefield.create_openmm_system(off_top)
integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('Reference')

/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [5]:
# create simulation
simulation = openmm.app.Simulation(pdb_file.topology, system, integrator, platform)

In [ ]:
# set initial positions from pdbfile
positions = pdb_file.getPositions()
simulation.context.setPositions(positions)

In [13]:
# set reporters
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', 1)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    1,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [27]:
simulation.context.setPositions(positions)
simulation.saveState('initial')
orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
#state_data_reporter.report(simulation, simulation.context.getState())
print('Initial Energy ' + str(orig_potential))
print('Minimizing Energy!')
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True, getForces=True)
min_potential = min_state.getPotentialEnergy()
print('Final Energy = ' + str(min_potential))


Initial Energy 11890.525741791453 kJ/mol
Minimizing Energy!
Final Energy = 7801.333976616224 kJ/mol


In [15]:
# try running a 0 step
simulation.step(1)


In [16]:
import mdtraj

initial = mdtraj.load_pdb('1100249_supercell.pdb')
final = mdtraj.load_pdb('trajectory.pdb')
rmsd = mdtraj.rmsd(initial,final)
print(rmsd[0])

0.014685853


In [35]:
# Need block to feed parameters to minimizer, where forces provide 3*n derivatives of energy
# wrt position, +3 more derivatives of energy wrt box vectors
# This block is a work in progress

def box_energy(x,*args):
    # x is np array of 3*n(atoms) positional coordinates and 6 coordinates of 3 triclinical box vectors
    # Return the energy of the system (in kJ/mol)
    # *args will have the simulation context and n (number of particles)
    context = args[0]
    n = args[1]
    # Build position array
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][0] = x[i*3]
        positions_arr[i][1] = x[i*3+1]
        positions_arr[i][2] = x[i*3+2]
    # Build periodic box vectors
    #a = np.array([x[n*3],0,0])
    #b = np.array([x[n*3+1],x[n*3+2],0])
    #c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    # Set Context with positions and periodic boundary conditions
    context.setPositions(positions_arr)
    #print(positions_arr)
    #context.setPeriodicBoxVectors(a,b,c)
    # Return Energy
    energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
    print(energy)
    return energy

def jacobian(x,*args):
    #must return a n*3 + 6 size vector with derivative of energy with respect to each input parameter
    #for positions, return given forces
    context = args[0]
    n = args[1]

    energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)

    # Positions
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][:] = x[i*3:i*3+3]
    context.setPositions(positions_arr)
    # box vectors
    #a = np.array([x[n*3],0,0])
    #b = np.array([x[n*3+1],x[n*3+2],0])
    #c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    epsilon = 1e-8
    # a_x
    temp_x = x
    #temp_x[n*3] = x[n*3] + epsilon
    # TODO: rescale positions?
    #new_energy = box_energy(temp_x,context,n)
    #grad_a_x = (new_energy-energy)/epsilon
    forces = -1*context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
    return forces.flatten()

In [17]:
# Dummy variables to test
test_state = simulation.context.getState(getPositions=True)
test_positions = test_state.getPositions(asNumpy=True)
x_test = test_positions.flatten()
n_test = len(test_positions)
test = jacobian(x_test,simulation.context,n_test)
print(test)

[  400.83722812 -1502.18623583  -814.53757672 ...   -20.18745337
   107.44029713   207.66900676]
[  400.83722812 -1502.18623583  -814.53757672 ...   -20.18745337
   107.44029713   207.66900676]


In [37]:
result = minimize(box_energy,x_test,(simulation.context,n_test),method='L-BFGS-B',jac=jacobian,options={'maxiter':200,'eps':1.0e-4})
print(result)


11890.525741791453
315261.99352872476
9707.688060294733
9197.092893032925
8786.380022326306
8564.454997537807
8395.293318736885
8300.483661258671
8235.658611625
8193.119757050572
8164.5649554014635
8091.735447453364
8027.410806067901
7946.787198851995
7887.492056798164
7838.6160709572105
7786.120917097875
7752.856557939624
7712.476072126381
7672.732359989404
7646.156791313935
7627.814509494283
7574.497567299666
7558.257700487447
7527.798234238635
7488.130191625733
7466.963070935459
7427.421196989611
7407.699232336214
7378.035889363182
7385.5577547760495
7357.983770059833
7329.544199703687
7305.517055580254
7286.394006472627
7268.852281765756
7254.540053141496
7241.690368614511
7231.7331971380245
7213.87583447004
7211.818341215316
7192.435117529465
7183.186553889471
7167.175592590349
7146.047794206481
7170.221599674263
7131.698687295728
7102.439606628412
7084.0019430189695
7066.87818169775
7051.4833589691625
7029.748718056526
7010.8777612557515
7000.20924422447
6981.737769205789
6963.44

In [7]:
# Example for tranformation of cartesian coordinates to fractional
pdb_file.topology.getUnitCellDimensions()
box_vectors = pdb_file.topology.getPeriodicBoxVectors()
print(box_vectors.value_in_unit(unit.nano*unit.meter))
A= np.array(box_vectors.value_in_unit(unit.nano*unit.meter))
print(A)
A_inv = np.linalg.inv(A)
x = np.array([[1,1,1],[1,1,1]])
y = np.matmul(A_inv,x.T).T
print(A_inv)
print(y)

25.62299919128418


In [8]:
positions = min_state.getPositions(asNumpy=True)
positions = positions.value_in_unit(unit.nano*unit.meter)
forces = min_state.getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
print(forces)
frac_positions = np.matmul(A_inv,positions.T).T
print(positions)


NameError: name 'min_state' is not defined

In [ ]:
print(box_vectors)
box_vector_array = np.array(box_vectors.value_in_unit(unit.nano*unit.meter)).flatten()
print(box_vector_array)
box_vector_array = np.delete(box_vector_array,[1,2,5]) #a_x,b_x,b_y,c_x,c_y,c_z
print(box_vector_array)
print(len(positions)/3)
